# Preprocessing data
This notebook prepares the data resulting from the FSRED post-processing notebook in order to plot it and perform clustering and MCMC analysis on it. 

In [1]:
import pandas as pd
import numpy as np

In [2]:
# Average observations of sources if they are in the same band on the same day
AVG = False

In [3]:
# Load in data
tbl = pd.read_csv('FSRED Mags - Final_python.csv')
display(tbl.head())

,Source,Filter,Obs_number,Detected Name,Date obs,RA,DEC,Error_circle (arcsec),Position_source,nH,...,Xray_Flux (erg/s/cm2),Xray_errup,Xray_errlow,Xray_upplim,Xray_Flux_source,L_X,L_X_errup,L_X_errlow,L_X_upplim,Comments
0,RX_J1735.3-3540,J,3081,S,2014-05-08,17 35 23.75,-35 40 16.1,0.56,"Israel+2008, UVOT",5.760000e+21,...,2.000000e-11,5.000000e-12,5.000000e-12,NaN,Thesis Bilal 2018,NaN,NaN,NaN,NaN,NaN
1,RX_J1735.3-3540,J,3090,S,2014-05-08,17 35 23.75,-35 40 16.1,0.56,"Israel+2008, UVOT",5.760000e+21,...,2.000000e-11,5.000000e-12,5.000000e-12,NaN,Thesis Bilal 2018,NaN,NaN,NaN,NaN,NaN
2,RX_J1735.3-3540,H,3063,S,2014-05-08,17 35 23.75,-35 40 16.1,0.56,"Israel+2008, UVOT",5.760000e+21,...,2.000000e-11,5.000000e-12,5.000000e-12,NaN,Thesis Bilal 2018,NaN,NaN,NaN,NaN,Initially very large magnitude offset but seem...
3,RX_J1735.3-3540,H,3072,S,2014-05-08,17 35 23.75,-35 40 16.1,0.56,"Israel+2008, UVOT",5.760000e+21,...,2.000000e-11,5.000000e-12,5.000000e-12,NaN,Thesis Bilal 2018,NaN,NaN,NaN,NaN,Initially large magnitude offset but seems oka...
4,RX_J1735.3-3540,Ks,3045,S,2014-05-08,17 35 23.75,-35 40 16.1,0.56,"Israel+2008, UVOT",5.760000e+21,...,2.000000e-11,5.000000e-12,5.000000e-12,NaN,Thesis Bilal 2018,NaN,NaN,NaN,NaN,Initially large magnitude offset but seems oka...


In [4]:
print(len(tbl.groupby(['Source'])))

38


### Handle fluxes

In [5]:
# Determine average flux 
avg_try = tbl.groupby(['Source', 'Filter', 'Date obs'], as_index=False).apply(lambda x : x['Flux (erg/s/cm2)'].mean())
mergetbl = pd.merge(tbl, avg_try, on=['Source', 'Filter', 'Date obs'])

# Replace flux by average value 
mergetbl['Flux (erg/s/cm2)'] = mergetbl[mergetbl.columns[-1]]

# Drop duplicate observations
avgtbl1 = mergetbl.drop_duplicates(subset=['Source', 'Filter', 'Date obs'], keep="first")


### Handle flux errors

In [6]:
# Determine average flux error  
avg_try2 = tbl.groupby(['Source', 'Filter', 'Date obs'], as_index=False).apply(lambda x : np.sqrt(np.sum(np.square(x['Flux_err']))))
mergetbl2 = pd.merge(tbl, avg_try2, on=['Source', 'Filter', 'Date obs'])

# Replace flux error by average value 
mergetbl2['Flux_err'] = mergetbl2[mergetbl2.columns[-1]]

# Drop duplicate observations
avgtbl2 = mergetbl2.drop_duplicates(subset=['Source', 'Filter', 'Date obs'], keep="first")

# Overwrite errors with average errors
avgtbl1['Flux_err'] = avgtbl2['Flux_err']

if AVG == True:
    tbl = avgtbl1

C:\Users\Iris Reitsma\AppData\Local\Temp\ipykernel_12316\229253048.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  avgtbl1['Flux_err'] = avgtbl2['Flux_err']


### Handle luminosity and errors

In [7]:
# TODO determine luminosities based on flux and distance
def luminosity(flux, d):
    d_cm = 3.08567758128e21 * float(d) 
    L = 4 * np.pi * d_cm**2 * flux
    
    return L

def luminosity_error(flux, flux_error, d, d_error, L):
    L_err = np.sqrt((flux_error/flux)**2 + 2*(d_error/float(d))**2) * L

    return L_err

# Determine NIR luminosity
tbl['L_NIR'] = tbl.apply(lambda x: luminosity(x['Flux (erg/s/cm2)'], x['Distance (kpc)']), axis=1)
tbl['L_NIR_errup'] = tbl.apply(lambda x: luminosity_error(x['Flux (erg/s/cm2)'], x['Flux_err'], x['Distance (kpc)'], x['D_errup'], x['L_NIR']), axis=1)
tbl['L_NIR_errlow'] = tbl.apply(lambda x: luminosity_error(x['Flux (erg/s/cm2)'], x['Flux_err'], x['Distance (kpc)'], x['D_errlow'], x['L_NIR']), axis=1)
tbl['L_NIR_errup_nodist'] = tbl.apply(lambda x: luminosity_error(x['Flux (erg/s/cm2)'], x['Flux_err'], x['Distance (kpc)'], 0, x['L_NIR']), axis=1)
tbl['L_NIR_errlow_nodist'] = tbl.apply(lambda x: luminosity_error(x['Flux (erg/s/cm2)'], x['Flux_err'], x['Distance (kpc)'], 0, x['L_NIR']), axis=1)
tbl['L_NIR_errup_noflux'] = tbl.apply(lambda x: luminosity_error(x['Flux (erg/s/cm2)'], 0, x['Distance (kpc)'], x['D_errup'], x['L_NIR']), axis=1)
tbl['L_NIR_errlow_noflux'] = tbl.apply(lambda x: luminosity_error(x['Flux (erg/s/cm2)'], 0, x['Distance (kpc)'], x['D_errlow'], x['L_NIR']), axis=1)

# Determine X-ray luminosity
tbl['L_X'] = tbl.apply(lambda x: luminosity(x['Xray_Flux (erg/s/cm2)'], x['Distance (kpc)']), axis=1)
tbl['L_X_errup'] = tbl.apply(lambda x: luminosity_error(x['Xray_Flux (erg/s/cm2)'], x['Xray_errup'], x['Distance (kpc)'], x['D_errup'], x['L_X']), axis=1)
tbl['L_X_errlow'] = tbl.apply(lambda x: luminosity_error(x['Xray_Flux (erg/s/cm2)'], x['Xray_errlow'], x['Distance (kpc)'], x['D_errlow'], x['L_X']), axis=1)
tbl['L_X_errup_nodist'] = tbl.apply(lambda x: luminosity_error(x['Xray_Flux (erg/s/cm2)'], x['Xray_errup'], x['Distance (kpc)'], 0, x['L_X']), axis=1)
tbl['L_X_errlow_nodist'] = tbl.apply(lambda x: luminosity_error(x['Xray_Flux (erg/s/cm2)'], x['Xray_errlow'], x['Distance (kpc)'], 0, x['L_X']), axis=1)
tbl['L_X_errup_noflux'] = tbl.apply(lambda x: luminosity_error(x['Xray_Flux (erg/s/cm2)'], 0, x['Distance (kpc)'], x['D_errup'], x['L_X']), axis=1)
tbl['L_X_errlow_noflux'] = tbl.apply(lambda x: luminosity_error(x['Xray_Flux (erg/s/cm2)'], 0, x['Distance (kpc)'], x['D_errlow'], x['L_X']), axis=1)

display(tbl.head())

,Source,Filter,Obs_number,Detected Name,Date obs,RA,DEC,Error_circle (arcsec),Position_source,nH,...,L_X_upplim,Comments,L_NIR_errup_nodist,L_NIR_errlow_nodist,L_NIR_errup_noflux,L_NIR_errlow_noflux,L_X_errup_nodist,L_X_errlow_nodist,L_X_errup_noflux,L_X_errlow_noflux
0,RX_J1735.3-3540,J,3081,S,2014-05-08,17 35 23.75,-35 40 16.1,0.56,"Israel+2008, UVOT",5.760000e+21,...,NaN,NaN,1.932908e+32,1.932908e+32,NaN,NaN,5.399185e+34,5.399185e+34,NaN,NaN
1,RX_J1735.3-3540,J,3090,S,2014-05-08,17 35 23.75,-35 40 16.1,0.56,"Israel+2008, UVOT",5.760000e+21,...,NaN,NaN,1.943706e+32,1.943706e+32,NaN,NaN,5.399185e+34,5.399185e+34,NaN,NaN
2,RX_J1735.3-3540,H,3063,S,2014-05-08,17 35 23.75,-35 40 16.1,0.56,"Israel+2008, UVOT",5.760000e+21,...,NaN,Initially very large magnitude offset but seem...,2.127279e+32,2.127279e+32,NaN,NaN,5.399185e+34,5.399185e+34,NaN,NaN
3,RX_J1735.3-3540,H,3072,S,2014-05-08,17 35 23.75,-35 40 16.1,0.56,"Israel+2008, UVOT",5.760000e+21,...,NaN,Initially large magnitude offset but seems oka...,2.462028e+32,2.462028e+32,NaN,NaN,5.399185e+34,5.399185e+34,NaN,NaN
4,RX_J1735.3-3540,Ks,3045,S,2014-05-08,17 35 23.75,-35 40 16.1,0.56,"Israel+2008, UVOT",5.760000e+21,...,NaN,Initially large magnitude offset but seems oka...,2.019295e+32,2.019295e+32,NaN,NaN,5.399185e+34,5.399185e+34,NaN,NaN


### Handle upper limits

In [8]:
def calc_flux(upplim, band):
    """ 
    Calculate flux density based on AB magnitude system
    using     m_AB = -2.5 log10(F_nu) - 48.60 
    and       F_lamb = F_nu * c / lamb_eff
    and convert this flux density to flux. 
    """ 
    
    band = band.strip() # remove accidental whitespace     
    
    # Calculate flux density 
    if band == 'J':
        W_eff = 2214.62 # A
        l_eff = 12287.26 # A
    elif band == 'H':
        W_eff = 2769.45 # A 
        l_eff = 16039.55 # A
    elif band == 'Ks':
        W_eff = 3163.40 # A
        l_eff = 21315.89 # A

    c = 2.9979e18 # A/s

    F_nu = 10**(-(upplim+48.60)/2.5) # erg/s/cm2/Hz   
    F_lamb = F_nu * c / l_eff**2 # erg/s/cm2/A
    Flux = W_eff * F_lamb 

    return Flux

# Determine upper limits on luminosities where needed
tbl['F_NIR_upplim'] = tbl.apply(lambda x: calc_flux(x['Upper_limit'], x['Filter']), axis=1)
tbl['L_NIR_upplim'] = tbl.apply(lambda x: luminosity(x['F_NIR_upplim'], x['Distance (kpc)']), axis=1)
tbl['L_X_upplim'] = tbl.apply(lambda x: luminosity(x['Xray_upplim'], x['Distance (kpc)']), axis=1)

### Add manual sources

In [9]:
# Manual adjustments for when only X-ray luminosity or upper limit on luminosity is known 
tbl.loc[tbl['Source']=='4U_1822-00', ['L_X']] = 1.80E+36
tbl.loc[tbl['Source']=='Swift_J1922.7-1716', ['L_X']] = 6.00E+31


### Transform luminosities to log

In [10]:
'''Take logs of luminosities''' 
# Luminosities
tbl['log_L_NIR'] = np.log10(tbl['L_NIR'])
tbl['log_L_X'] = np.log10(tbl['L_X'])

# Errors
tbl['log_L_NIR_errup'] = 1/np.log(10) * tbl['L_NIR_errup'] / tbl['L_NIR']
tbl['log_L_NIR_errlow'] = 1/np.log(10) * tbl['L_NIR_errlow'] / tbl['L_NIR']
tbl['log_L_X_errup'] = 1/np.log(10) * tbl['L_X_errup'] / tbl['L_X']
tbl['log_L_X_errlow'] = 1/np.log(10) * tbl['L_X_errlow'] / tbl['L_X']

tbl['log_L_NIR_errup_nodist'] = 1/np.log(10) * tbl['L_NIR_errup_nodist'] / tbl['L_NIR']
tbl['log_L_NIR_errlow_nodist'] = 1/np.log(10) * tbl['L_NIR_errlow_nodist'] / tbl['L_NIR']
tbl['log_L_NIR_errup_noflux'] = 1/np.log(10) * tbl['L_NIR_errup_noflux'] / tbl['L_NIR']
tbl['log_L_NIR_errlow_noflux'] = 1/np.log(10) * tbl['L_NIR_errlow_noflux'] / tbl['L_NIR']

tbl['log_L_X_errup_nodist'] = 1/np.log(10) * tbl['L_X_errup_nodist'] / tbl['L_X']
tbl['log_L_X_errlow_nodist'] = 1/np.log(10) * tbl['L_X_errlow_nodist'] / tbl['L_X']
tbl['log_L_X_errup_noflux'] = 1/np.log(10) * tbl['L_X_errup_noflux'] / tbl['L_X']
tbl['log_L_X_errlow_noflux'] = 1/np.log(10) * tbl['L_X_errlow_noflux'] / tbl['L_X']

# upper limits
tbl['log_L_NIR_upplim'] = np.log10(tbl['L_NIR_upplim'])
tbl['log_L_X_upplim'] = np.log10(tbl['L_X_upplim'])

### Print statistics

In [11]:
print(np.std(tbl['D_errup']/tbl['Distance (kpc)']))
print(np.mean(tbl['D_errup']/tbl['Distance (kpc)']))

0.13842767245405735
0.21069057726300147


In [12]:
print(np.mean(tbl['log_L_NIR_errup']))
print(np.mean(tbl['log_L_X_errup']))

0.13801648233186792
0.15496230547719425


### Store dataframe

In [13]:
if AVG == True: 
    tbl.to_csv('preprocessed_data_avg.csv')
else:
    tbl.to_csv('preprocessed_data.csv')